In [1]:
import astrodetection
import glob
import pandas as pd
import os
import numpy as np

In [ ]:
path = "PATH_TO_FOLDER"

In [3]:
df_dict = {}
for file in glob.glob(f"{path}/*.json"):
    df = pd.read_json(file)
    df.index = df.index.astype(str) #compatibility with d3lta
    n_file = os.path.split(file)[-1]
    df_dict[n_file] = df

In [4]:
def preprocess_df(df):

  df = df[df['tweet'].str.len() > 100]
  df = df[df['username']!='grok']
  df.index = df.index.astype(str)

  return df

In [5]:
for n_file, df in df_dict.items():
    df_dict[n_file] = preprocess_df(df)

In [7]:
dict_matches = {}
dict_scores = {}

for n_file, df in df_dict.items():
    df_filtered, df_emb = astrodetection.prepare_input_data(df, 
        embeddings=df['emb'])
    
    matches, df_cluster = astrodetection.semantic_faiss(
        df_filtered.rename(columns={'tweet':'original'}),
        min_size_txt = 0,
        df_embeddings_use = df_emb,
        threshold_grapheme = 0.8,
        threshold_language = 0.715,
        threshold_semantic = 0.9
    )

    dict_matches[n_file] = matches

    dict_scores[n_file] = astrodetection.compute_bot_likelihood_metrics(
        df,
        matches=matches
    )

>>> Start prepare_dataset
Done.

Removing 0 short texts over 3464 sentences...
Done.
<<< End prepare_dataset, Took: 9.7231 sec
>>> Start create_index_cosine
<<< End create_index_cosine, Took: 0.0143 sec
>>> Start find_matches


  0%|          | 0/35 [00:00<?, ?it/s]

<<< End find_matches, Took: 3.1344 sec
>>> Start compute_duplicate_types
<<< End compute_duplicate_types, Took: 1.2434 sec
>>> Start prepare_dataset
Done.

Removing 0 short texts over 4270 sentences...
Done.
<<< End prepare_dataset, Took: 11.6766 sec
>>> Start create_index_cosine
<<< End create_index_cosine, Took: 0.0118 sec
>>> Start find_matches


  0%|          | 0/43 [00:00<?, ?it/s]

<<< End find_matches, Took: 9.3723 sec
>>> Start compute_duplicate_types
<<< End compute_duplicate_types, Took: 4.5779 sec
>>> Start prepare_dataset
Done.

Removing 0 short texts over 3463 sentences...
Done.
<<< End prepare_dataset, Took: 9.9415 sec
>>> Start create_index_cosine
<<< End create_index_cosine, Took: 0.0043 sec
>>> Start find_matches


...

<<< End find_matches, Took: 0.2661 sec
>>> Start compute_duplicate_types
<<< End compute_duplicate_types, Took: 0.0417 sec
>>> Start prepare_dataset
Done.

Removing 0 short texts over 917 sentences...
Done.
<<< End prepare_dataset, Took: 2.5723 sec
>>> Start create_index_cosine
<<< End create_index_cosine, Took: 0.0011 sec
>>> Start find_matches


  0%|          | 0/10 [00:00<?, ?it/s]

<<< End find_matches, Took: 1.3378 sec
>>> Start compute_duplicate_types
<<< End compute_duplicate_types, Took: 0.6104 sec
>>> Start prepare_dataset
Done.

Removing 0 short texts over 2153 sentences...
Done.
<<< End prepare_dataset, Took: 5.2120 sec
>>> Start create_index_cosine
<<< End create_index_cosine, Took: 0.0056 sec
>>> Start find_matches


  0%|          | 0/22 [00:00<?, ?it/s]

<<< End find_matches, Took: 0.6525 sec
>>> Start compute_duplicate_types
<<< End compute_duplicate_types, Took: 0.1280 sec


In [8]:
df_scores = pd.DataFrame.from_dict(dict_scores, orient='index')

In [9]:
def colora_cella(val, valore_riferimento):
    if pd.isna(val):
        return ''
    if val > 2 * valore_riferimento:
        return 'background-color: salmon'
    elif val > valore_riferimento:
        return 'background-color: yellow'
    else:
        return 'background-color: lightgreen'


valori_riferimento = {}
for col in df_scores.columns:
    valori_riferimento[col] = max(df_scores.loc['lundi_semaine.json', col], df_scores.loc['OmarSy.json', col])

def applica_colorazione(row):
    styled_row = []
    for col, val in row.items():
        if col != 'number_of_tweets':
          styled_row.append(colora_cella(val, valori_riferimento[col]))
        else:
          styled_row.append('')
    return styled_row

df_scores_colorato = df_scores.style.apply(applica_colorazione, axis=1)

In [10]:
df_scores_colorato

,copypasta_score (%),top_users_post_percent (%),top_users_count,zero_followers_and_following (%),no_image_and_description (%),default_handle_score (%),number_of_tweets
punaises_de_lit.json,9.500000,5.660000,29,0.060000,5.280000,8.340000,3464
penurie.json,12.950000,12.150000,25,0.420000,1.870000,5.480000,4270
lundi_semaine.json,4.790000,5.310000,30,0.090000,2.110000,7.620000,3463
ZFE.json,40.430000,11.130000,5,0.160000,1.800000,4.750000,611
trogneux_1000.json,23.020000,6.160000,12,0.220000,3.960000,12.760000,1364
OmarSy.json,9.960000,7.750000,10,0.210000,1.490000,4.770000,1406
elnet.json,15.450000,12.270000,2,0.450000,4.090000,15.450000,220
cadmium_long.json,23.560000,7.850000,7,0.000000,2.840000,11.780000,917
shogun.json,9.570000,23.780000,10,0.050000,0.880000,6.180000,2153


In [11]:
astrodetection.create_network(dict_matches['cadmium_long.json'], df_dict['cadmium_long.json'])

Sigma(nx.DiGraph with 216 nodes and 479 edges)